In [1]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
import pandas as pd
from datetime import timedelta, datetime
from attendance_prediction_model.new_attedance_pred import predict_attendance

dashboard_utils = DashboardUtils()

In [5]:
current_date = datetime.today()
start_date = current_date - timedelta(days=5)
end_date = datetime.strptime('2025-02-16', '%Y-%m-%d')

dashboard_utils.data.attendance = dashboard_utils.data.load_file('attendance.csv')
dashboard_utils.data.clean_attendance()
dashboard_utils.data.preprocess_attendance()

hist = dashboard_utils.data.attendance
hist['USAGE_DATE'] += pd.Timedelta(days=365*3+1)
hist = hist[(hist['USAGE_DATE'] <= current_date) & (hist['USAGE_DATE'] >= start_date)].reset_index(drop=True)
hist['predicted'] = 0

hist

,USAGE_DATE,attendance,predicted
0,2025-02-09,20363,0
1,2025-02-10,16755,0
2,2025-02-11,18223,0
3,2025-02-12,30480,0
4,2025-02-13,27346,0


In [7]:
hist, pred = dashboard_utils.get_predicted_attendance_with_past(current_date, start_date)

hist

/Users/hadrienmorand/Documents/Studies/ESSEC_CENTRALE/Studies/M2/T2/Centrale/Hackathon/Hackathon-ElevenStrategy/endless_line/data_utils/weather_forecast.py:73: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_df = forecast_df.set_index('dt_iso').resample('h').interpolate(method='ffill', limit_direction='forward')


,USAGE_DATE,attendance,predicted
0,2025-02-09,20363,0
1,2025-02-10,16755,0
2,2025-02-11,18223,0
3,2025-02-12,30480,0
4,2025-02-13,27346,0


In [9]:
import plotly.graph_objects as go

if datetime.today().hour > 12:
    # Manually add transition point to keep continuity
    transition_x = [hist["USAGE_DATE"].iloc[-1], pred["USAGE_DATE"].iloc[0]]
    transition_y = [hist["attendance"].iloc[-1], pred["attendance"].iloc[0]]

fig = go.Figure()

# Historical data (Blue Line)
fig.add_trace(go.Scatter(
    x=hist["USAGE_DATE"],
    y=hist["attendance"],
    mode="lines",
    line=dict(color="blue", width=2),
    line_shape="spline",
    name="Historical Attendance"
))

# Transition Line (Connect Last Historical to First Prediction)
fig.add_trace(go.Scatter(
    x=transition_x,
    y=transition_y,
    mode="lines",
    line=dict(color="blue", width=2, dash="dot"),
    line_shape="spline",
    showlegend=False
))

# Predicted Data (Red Line)
fig.add_trace(go.Scatter(
    x=pred["USAGE_DATE"],
    y=pred["attendance"],
    mode="lines",
    line=dict(color="red", width=2),
    line_shape="spline",
    name="Predicted Attendance"
))

# Layout enhancements
fig.update_layout(
    title="Attendance Forecast",
    xaxis_title="Date",
    yaxis_title="Attendance",
    hovermode="x unified",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor="white"
)

fig.show()
